In [15]:
import numpy as np
import keras
import keras.backend as K
import tensorflow as tf
from keras.layers import *
from keras.models import *

C:\Users\acyrl\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Guide to [writing your own keras layer](https://keras.io/layers/writing-your-own-keras-layers/).

In [38]:
class NAC(Layer):
    
    def __init__(self,
                 units,
                 W_hat_initializer='glorot_uniform', # need to change default initializer.
                 M_hat_initializer='glorot_uniform', # need to change default initializer.
                 name=None,
                 **kwargs
                ):
        
        super(NAC, self).__init__(**kwargs)
        
        self.units = units
        self.W_hat_initializer = initializers.get(W_hat_initializer)
        self.M_hat_initializer = initializers.get(M_hat_initializer)
    
    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        self.W_hat = self.add_weight(shape=(input_dim, self.units),
                                     initializer=self.W_hat_initializer,
                                     name='W_hat')
        self.M_hat = self.add_weight(shape=(input_dim, self.units),
                                     initializer=self.M_hat_initializer,
                                     name='M_hat')
        self.built = True
    
    def call(self, inputs):
        W = K.tanh(self.W_hat) * K.sigmoid(self.M_hat)
        return K.dot(inputs, W)

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def get_config(self):
        config = {
            'units': self.units,
            'W_hat_initializer': initializers.serialize(self.W_hat_initializer),
            'M_hat_initializer': initializers.serialize(self.M_hat_initializer)
        }
        base_config = super(NAC, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [23]:
import GenData as gd
X,Y = gd.gd_paper(size=10000)
valX, valY = gd.gd_paper(size=50)

[ 66.44574769 140.28945204] [206.73519973]


In [ ]:
split = 45
trX = np.random.normal(0, 0.5, (1000, 100))
a = trX[:, :split].sum(1)
b = trX[:, split:].sum(1)

#x = keras.layers.InputSpec()

x = keras.layers.Input((10,))
y = NALUcell(1, kernel_initializer=initializers.Constant(10))(x)

In [ ]:
model.fit(a, b, batch_size=20, epochs=20)

In [17]:
x

<tf.Tensor 'input_6:0' shape=(?, 10) dtype=float32>

In [48]:
print(X.shape, Y.shape)
np.random.rand(128, 10).shape

(128, 2) (128, 1)


(128, 10)

In [49]:
x = Input((1,)) # n is the dimension of the data, (n, N)
y = NAC(1, W_hat_initializer=initializers.truncated_normal())(x) # number of units, but let's test
m = Model(x, y)
m.compile(keras.optimizers.RMSprop(lr=0.01), "mse", metrics=['accuracy'])
m.fit(Y, Y, batch_size=20, epochs=1, verbose=1)

Epoch 1/1
10000/10000 [==============================] - 1s 95us/step - loss: 5198.4618 - acc: 1.0000e-04


In [69]:
t = np.array([2,2])
print(X[1:10])
m.predict(X[1:10])


[[ 37.91400752   8.57856319]
 [ 91.05914504 100.00486347]
 [ 65.71871739  13.33499236]
 [ 35.96730318  91.18802709]
 [ 58.76270934  24.79102971]
 [167.62736609  10.67597158]
 [ 19.18224718  74.13619536]
 [100.13142436  40.00189752]
 [ 49.19071227 106.76310138]]


array([[ 46.40562 ],
       [190.7091  ],
       [ 78.90582 ],
       [126.920074],
       [ 83.39783 ],
       [177.96904 ],
       [ 93.14606 ],
       [139.87178 ],
       [155.66507 ]], dtype=float32)

In [41]:
Xtest, Ytest = gd.gd_paper(lower_bound=10, upper_bound=50, size=100)

In [52]:
class NALU(Layer):
    
    def __init__(self,
                 units,
                 W_hat_initializer = 'glorot_uniform',
                 M_hat_initializer = 'glorot_uniform',
                 G_initializer = 'glorot_uniform',
                 epsilon=1e-7,
                 name=None,
                 **kwargs
                ):
        
        super(NALU, self).__init__(**kwargs)
        
        self.units = units
        self.W_hat_initializer = initializers.get(W_hat_initializer)
        self.M_hat_initializer = initializers.get(M_hat_initializer)
        self.G_initializer = initializers.get(G_initializer)
        self.epsilon = epsilon
    
    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        self.W_hat = self.add_weight(shape=(input_dim, self.units),
                                     initializer=self.W_hat_initializer,
                                     name='W_hat')
        self.M_hat = self.add_weight(shape=(input_dim, self.units),
                                     initializer=self.M_hat_initializer,
                                     name='M_hat')
        self.G = self.add_weight(shape=(input_dim, self.units),
                             initializer=self.G_initializer,
                             name='G')
        self.built = True
    
    def call(self, inputs):
        W = K.tanh(self.W_hat) * K.sigmoid(self.M_hat)
        a = K.dot(inputs, W)
        m = K.exp(K.dot(K.log(K.abs(inputs) + self.epsilon), W))
        g = K.sigmoid(K.dot(x, self.G))
        y = g*a + (1-g)*m
        
        return y

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def get_config(self):
        config = {
            'units': self.units,
            'W_hat_initializer': initializers.serialize(self.W_hat_initializer),
            'M_hat_initializer': initializers.serialize(self.M_hat_initializer),
            'G_initializer': initializers.serialize(self.M_hat_initializer),
            'epsilon': self.epsilon,
        }
        base_config = super(NALU, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [53]:
x = Input((1,)) # n is the dimension of the data, (n, N)
y = NALU(1)(x) # number of units, but let's test
m = Model(x, y)
m.compile(keras.optimizers.RMSprop(lr=0.01), "mse", metrics=['accuracy'])
m.fit(Y, Y, batch_size=20, epochs=1, verbose=1)

Epoch 1/5
10000/10000 [==============================] - 1s 115us/step - loss: 30331.9581 - acc: 0.0000e+00
Epoch 2/5
10000/10000 [==============================] - 1s 92us/step - loss: 1351.3113 - acc: 1.0000e-04
Epoch 3/5
10000/10000 [==============================] - 1s 92us/step - loss: 0.0559 - acc: 1.0000e-04
Epoch 4/5
10000/10000 [==============================] - 1s 94us/step - loss: 1.6378e-05 - acc: 1.0000e-04
Epoch 5/5
10000/10000 [==============================] - 1s 89us/step - loss: 1.7990e-08 - acc: 1.0000e-04
